# Obesity Predictor — Interactive UI
This notebook trains the Random Forest and provides an ipywidgets form to predict obesity levels.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load data
df = pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv")

# Encode categoricals
label_encoders = {}
for col in df.select_dtypes(include="object").columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

X = df.drop("NObeyesdad", axis=1)
y = df["NObeyesdad"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Create and train Random Forest 
rf_model = RandomForestClassifier(
    n_estimators=100,        # Number of decision trees
    max_features="sqrt",     # Random feature selection
    bootstrap=True,          # Bagging (Bootstrap Aggregation)
    random_state=42
)

rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))

Random Forest Accuracy: 0.9527186761229315


In [ ]:
import os
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output

# Quiet the loky core-count warning on Windows
os.environ.setdefault("LOKY_MAX_CPU_COUNT", "4")
warnings.filterwarnings(
    "ignore",
    message="Could not find the number of physical cores",
    category=UserWarning
)

# Create form fields with labels above inputs
def create_field(label_text, widget):
    label = widgets.HTML(value=f"<b>{label_text}</b>")
    return widgets.VBox([label, widget], layout=widgets.Layout(margin='5px 0'))

# Input widgets without descriptions (labels will be separate)
form_widgets = {
    "Gender": widgets.Dropdown(options=[("Female", 0), ("Male", 1)], layout=widgets.Layout(width='300px')),
    "Age": widgets.BoundedIntText(value=25, min=1, max=120, layout=widgets.Layout(width='300px')),
    "Height": widgets.BoundedFloatText(value=1.70, min=1.0, max=3.0, step=0.01, layout=widgets.Layout(width='300px')),
    "Weight": widgets.BoundedFloatText(value=70.0, min=20.0, max=300.0, step=0.1, layout=widgets.Layout(width='300px')),
    "family_history_with_overweight": widgets.Dropdown(options=[("No", 0), ("Yes", 1)], layout=widgets.Layout(width='300px')),
    "FAVC": widgets.Dropdown(options=[("No", 0), ("Yes", 1)], layout=widgets.Layout(width='300px')),
    "FCVC": widgets.BoundedIntText(value=2, min=0, max=3, layout=widgets.Layout(width='300px')),
    "NCP": widgets.BoundedIntText(value=3, min=1, max=4, layout=widgets.Layout(width='300px')),
    "CAEC": widgets.BoundedIntText(value=1, min=0, max=3, layout=widgets.Layout(width='300px')),
    "SMOKE": widgets.Dropdown(options=[("No", 0), ("Yes", 1)], layout=widgets.Layout(width='300px')),
    "CH2O": widgets.BoundedIntText(value=2, min=1, max=10, layout=widgets.Layout(width='300px')),
    "SCC": widgets.Dropdown(options=[("No", 0), ("Yes", 1)], layout=widgets.Layout(width='300px')),
    "FAF": widgets.BoundedIntText(value=2, min=0, max=3, layout=widgets.Layout(width='300px')),
    "TUE": widgets.BoundedIntText(value=1, min=0, max=24, layout=widgets.Layout(width='300px')),
    "CALC": widgets.BoundedIntText(value=1, min=0, max=2, layout=widgets.Layout(width='300px')),
    "MTRANS": widgets.BoundedIntText(value=2, min=0, max=4, layout=widgets.Layout(width='300px')),
}

# Field labels matching console prompts
field_labels = {
    "Gender": "Gender (0=Female, 1=Male):",
    "Age": "Age:",
    "Height": "Height (in meters, e.g., 1.75):",
    "Weight": "Weight (in kg):",
    "family_history_with_overweight": "Family history with overweight? (0=No, 1=Yes):",
    "FAVC": "Frequently eat high caloric food? (0=No, 1=Yes):",
    "FCVC": "Frequency of vegetables (0-3):",
    "NCP": "Number of main meals per day (1-4):",
    "CAEC": "Eat between meals? (0=No, 1=Sometimes, 2=Frequently, 3=Always):",
    "SMOKE": "Do you smoke? (0=No, 1=Yes):",
    "CH2O": "Daily water consumption (liters, 1-10):",
    "SCC": "Monitor calories? (0=No, 1=Yes):",
    "FAF": "Physical activity frequency (0-3):",
    "TUE": "Time using technology (hours, 0-24):",
    "CALC": "Alcohol consumption (0=None, 1=Sometimes, 2=Frequently):",
    "MTRANS": "Transportation (0=Walking, 1=Bike, 2=Motorbike, 3=Public, 4=Car):",
}

# Create fields with labels above
form_fields = [create_field(field_labels[k], form_widgets[k]) for k in form_widgets.keys()]

# Remember original borders
_base_borders = {k: w.layout.border for k, w in form_widgets.items()}

def _set_error_border(name):
    form_widgets[name].layout.border = "2px solid #e55353"

def _clear_error_borders():
    for k, w in form_widgets.items():
        w.layout.border = _base_borders[k]

predict_button = widgets.Button(description="Predict Obesity Level", button_style="success")
status_bar = widgets.HTML(value="Ready to predict.")
output_area = widgets.Output()

num_fields_int = ["Gender", "Age", "family_history_with_overweight", "FAVC", "FCVC", "NCP", "CAEC", "SMOKE", "CH2O", "SCC", "FAF", "TUE", "CALC", "MTRANS"]
num_fields_float = ["Height", "Weight"]

bounds = {
    "Age": (1, 120),
    "Height": (1.0, 3.0),
    "Weight": (20.0, 300.0),
    "FCVC": (0, 3),
    "NCP": (1, 4),
    "CAEC": (0, 3),
    "CH2O": (1, 10),
    "FAF": (0, 3),
    "TUE": (0, 24),
    "CALC": (0, 2),
    "MTRANS": (0, 4),
}

def validate_and_cast():
    _clear_error_borders()
    errors = []
    casted = {}
    for label, widget in form_widgets.items():
        val = widget.value
        if val is None or (isinstance(val, str) and val.strip() == ""):
            errors.append(f"{label}: required")
            _set_error_border(label)
            continue
        if label in num_fields_int:
            try:
                casted[label] = int(val)
            except Exception:
                errors.append(f"{label}: must be an integer")
                _set_error_border(label)
                continue
        elif label in num_fields_float:
            try:
                casted[label] = float(val)
            except Exception:
                errors.append(f"{label}: must be a number")
                _set_error_border(label)
                continue
        else:
            casted[label] = val

        if label in bounds and label in casted:
            lo, hi = bounds[label]
            if not (lo <= casted[label] <= hi):
                errors.append(f"{label}: must be between {lo} and {hi}")
                _set_error_border(label)
    return errors, casted

def on_predict_clicked(_):
    status_bar.value = "Running prediction…"
    with output_area:
        clear_output()
        errors, casted = validate_and_cast()
        if errors:
            status_bar.value = "Fix validation errors"
            print("Validation issues:")
            for err in errors:
                print(" -", err)
            return
        try:
            user_input = pd.DataFrame({k: [v] for k, v in casted.items()})
            print("Inputs:")
            print(user_input.T)

            pred = rf_model.predict(user_input)
            proba = rf_model.predict_proba(user_input)[0]
            label = label_encoders["NObeyesdad"].inverse_transform(pred)[0]

            print("\n" + "=" * 60)
            print(f"Predicted Obesity Level: {label}")
            print("Confidence:")
            for idx, prob in enumerate(proba):
                class_label = label_encoders["NObeyesdad"].inverse_transform([idx])[0]
                bar = "" * int(prob * 40)
                print(f"  {class_label:25} {prob:6.2%} {bar}")
            print("=" * 60)
            status_bar.value = "Done"
        except Exception as exc:
            print("Prediction error:", exc)
            status_bar.value = "Error — see output"

# Clear any existing handlers and attach the new one
predict_button.on_click(on_predict_clicked, remove=True)
predict_button.on_click(on_predict_clicked)

form_layout = widgets.VBox(
    form_fields + [predict_button, status_bar, output_area]
)
display(form_layout)